In [1]:
import talib as ta

from datetime import datetime
import pandas as pd
import numpy as np
import pickle

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('data2_D1.csv', parse_dates=['Datetime'], index_col='Datetime')

In [3]:
df.head()

,Target,Returns,ADX_diff,CCI_diff,MOM_diff,RSI_diff,Linear_regression_diff,Linear_angle_diff,Linear_slope_diff,Linear_intercept_diff
Datetime,,,,,,,,,,
2003-09-07,0,-0.001145,-3.633067,12.798014,-0.00601,-0.792645,0.005093,0.034840,0.000608,-0.002812
2003-09-08,1,-0.002158,-3.080056,-46.153731,-0.00012,-1.534490,0.003217,0.016252,0.000284,-0.000471
2003-09-09,0,0.013599,-4.006817,31.678869,0.01236,8.554027,0.006306,0.032812,0.000573,-0.001139
2003-09-10,1,-0.003188,-3.720616,-16.883688,-0.01383,-2.405602,0.003946,0.012456,0.000217,0.001120
2003-09-11,1,0.000179,-3.083618,-21.011796,-0.00050,0.109947,0.003661,0.012806,0.000224,0.000756


In [4]:
end = datetime(2013,12,31)
test = datetime(2014,1,1)

X_train = df.loc[:end].drop('Target', axis=1).values
y_train = df.loc[:end].Target.values

X_test = df.loc[test:].drop('Target', axis=1).values
y_test = df.loc[test:].Target.values

In [5]:
steps = [
    ('poly', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=3, n_jobs=-1))
]

knn = Pipeline(steps)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_pred) * 100)

49.905956112852664


In [7]:
steps = [
    ('poly', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
]
params = {
    'knn__n_neighbors': np.arange(1, 20),
    'knn__n_jobs': [-1]
}
pipeline = Pipeline(steps)
knn_cv = GridSearchCV(pipeline, params, cv=5, n_jobs=-1, verbose=1)
knn_cv.fit(X_train, y_train)
print(knn_cv.best_params_)
print(knn_cv.best_score_)

model = knn_cv.best_estimator_

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('train: {}'.format(model.score(X_train, y_train) * 100))
print('test: {}'.format(model.score(X_test, y_test) * 100))
print('accuracy score: {}'.format(accuracy_score(y_test, y_pred) * 100))

with open('knn_data2_D1.pickle', 'wb') as file:
    pickle.dump(model, file)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  95 out of  95 | elapsed:   11.5s finished


{'knn__n_jobs': -1, 'knn__n_neighbors': 16}
0.5261528938409161
train: 61.77653977096874
test: 51.03448275862069
accuracy score: 51.03448275862069
